<a href="https://colab.research.google.com/github/NeagrTavakol/Final_dataminning/blob/main/Copy_of_Final_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Read dev dataset**

In [ ]:
dev_csv="/content/drive/MyDrive/Colab Notebooks/dev.csv"
dev_df = pd.read_csv(dev_csv,sep='\t')
dev_df=dev_df.drop("Unnamed: 0",axis=1)
dev_columns=dev_df.columns

**Read train dataset**

In [94]:
train_csv="/content/drive/MyDrive/Colab Notebooks/train.csv"
train_df = pd.read_csv(train_csv,sep='\t')
train_df=train_df.drop("Unnamed: 0",axis=1)
train_columns = train_df.columns

# **Tokenize Data**

**install packages**

In [ ]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import the package
import stanza

     |████████████████████████████████| 432 kB 4.3 MB/s 
     |████████████████████████████████| 174 kB 48.3 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=5956e340f2236efab06f4d9da57ce6bffb2447f632b8d35d356b9a1130df1643
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [ ]:
# Download an Persiabn model into the default directory
print("Downloading Persian model...")
stanza.download('fa')

2022-01-27 17:28:17 INFO: Downloading default packages for language: fa (Persian)...


2022-01-27 17:28:38 INFO: Finished downloading models and saved to /root/stanza_resources.


**create models**

In [ ]:
# Build a Persian pipeline, with all processors by default
print("Building a Persian pipeline...")
fa_nlp = stanza.Pipeline('fa')

2022-01-27 17:28:41 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-01-27 17:28:41 INFO: Use device: cpu
2022-01-27 17:28:41 INFO: Loading: tokenize


Building a Persian pipeline...


2022-01-27 17:28:41 INFO: Loading: mwt
2022-01-27 17:28:41 INFO: Loading: pos
2022-01-27 17:28:42 INFO: Loading: lemma
2022-01-27 17:28:42 INFO: Loading: depparse
2022-01-27 17:28:43 INFO: Done loading processors!


**find tokens of dataframe and save in array**

In [ ]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
tokenized_array=[]
for i in range(0,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

In [ ]:
#find tokens of rest of dataset (process interupted cause of volume of dataset)
# rest_train_df=train_df[52466:]
for i in range(52466,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


**save tokens array to csv file**

In [ ]:
#save tokens to array
import csv
with open('tokens_array.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(tokenized_array)

**convert tokens array to dataframe **

In [ ]:
# convert tokenized_array to dataframe
tokenized_df=pd.DataFrame(tokenized_array)

**save dataframe to csv file**

In [ ]:
#save dataframe 
tokenized_df.to_csv('tokens_df.csv')

In [ ]:
# with open('tokens_array.csv', 'r', newline='') as file:
#   myreader = csv.reader(file, delimiter=',')
#   for rows in myreader:
#     print(rows)

## **Read tokenized dataet**

**read from array csv file (smaller size)**

تو از اینجا ران کن(فایلش رو تلگرام فرستادم برات)

In [3]:
#read array csv file
tokens_array=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_array.csv",sep='\t',header=None)
# tokens_array.head()

## **Preproccessing**

**Remove punctuation**

In [4]:
from types import new_class
# from string import punctuation
import re

punctuation=['!','"','#','$','%','&',"'",'(',')','*','+','،','-','.',','"/",':',';','<','=','>','?','@','[',"\\","]","^",'_','`','{','|','}','~']
# tokens_array.to_numpy()
tokens_arr=[]
for i in tokens_array.to_numpy():
  sent=[]
  for j in i[0].split(','):
    if j not in punctuation:
      sent.append(j)
  tokens_arr.append(sent)

In [5]:
for i in tokens_arr[:10]:
  print(i)

['واقعا', 'حیف', 'وقت', 'که', 'بنویسم', 'سرویس', 'دهیتون', 'شده', 'افتضاح']
['قرار', 'بود', '۱', 'ساعته', 'برسه', 'ولی', 'نیم', 'ساعت', 'زودتر', 'از', 'موقع', 'رسید', 'شما', 'ببین', 'چقدرررررررررررر', 'پلاک', 'خفنههههه', 'من', 'سالها', 'ست', 'مشتریشونم', 'و', 'سالها', 'ست', 'مزه', 'بهشت', 'میده', 'غذاشون']
['قیمت', 'این', 'مدل', 'اصلا', 'با', 'کیفیتش', 'سازگاری', 'نداره', 'فقط', 'ظاهر', 'فریبنده', 'داره', 'پرش', 'میکنن', 'کالباس', 'و', 'قارچ']
['عالللی', 'بود', 'همه', 'چه', 'درست', 'و', 'به', 'اندازه', 'و', 'کیفیت', 'خوب', 'امیداورم', 'همیشه', 'کیفیتتون', 'خوب', 'باشه', 'ما', 'مشتری', 'همیشگی', 'بشیم']
['شیرینی', 'وانیلی', 'فقط', 'یک', 'مدل', 'بود']
['بدترین', 'پیتزایی', 'که', 'تا', 'به', 'حال', 'خورده', 'بودم']
['از', 'همه', 'لحاظ', 'عالی', 'ممنون', 'م']
['کیفیت', 'غذا', 'متوسط', 'رو', 'به', 'پایین', 'بود', 'انگار', 'داخل', 'یه', 'رستوران', 'معمولی', 'غذا', 'خوردی', 'درحالی', 'که', 'امتیاز', 'رستوران', 'در', 'اسنپ', 'فود', '۴٫۳', 'بود']
['همه', 'اقلام', 'تازه', 'و', 'به', 'روز', 'وخیل

**Analyze Reviews Length**

In [6]:
import matplotlib.pyplot as plt
reviews_len = [len(x) for x in tokens_arr]
reviews_len=np.array(reviews_len)

In [ ]:
# pd.Series(tokens_arr).hist()
# plt.show()
# pd.Series(tokens_arr).describe()

In [7]:
print("min:",reviews_len.min(),"max:",reviews_len.max(),"min:",reviews_len.mean(),"len:",len(reviews_len))

min: 3 max: 362 min: 19.003015873015872 len: 56700


In [8]:
# for i in range(6,29):
print(np.count_nonzero(reviews_len > 75))

689


**Remove outliers**

In [9]:
cleaned_rokens_arr=[x for x in tokens_arr if len(x)<75]
print(len(cleaned_rokens_arr))

55978


ثریا 
اینجا کد ورد امبدنیگ رو میتونی ببینی 
کلمات مختلفی هم باهاش امتحان کردم اوکی شد

## **Word embedding**

In [10]:
!pip install gensim
# !pip install nltk

In [ ]:
#for_test_for_word_embedding
# import gensim
# from gensim.models import Word2Vec
# from nltk.tokenize import sent_tokenize, word_tokenize
# import pandas as pd


# rev.head()
# corpus_text = 'n'.join(rev[:1000]['Text'])
# data = []
# # iterate through each sentence in the file
# for i in sent_tokenize(corpus_text):
#     temp = []
#     # tokenize the sentence into words
#     for j in word_tokenize(i):
#         temp.append(j.lower())
#     data.append(temp)

# word2vec= gensim.models.Word2Vec(data, min_count=5,size=100)
# word2vec = Word2Vec(cleaned_rokens_arr, min_count=5,size=100)
# کد بالایی ها رو فعلا بذار باشه و اینکه کامنت باشه  

In [11]:
#main_code
import gensim
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('/content/drive/MyDrive/Colab Notebooks/model.bin', binary=True)

In [ ]:
vector = model['خوب']
# see the shape of the vector (300,)
# vector.shape
vector

In [67]:
v=model['خوب']
len(v)

100

In [ ]:
model.most_similar('بهتر')

In [ ]:
model.most_similar('طعم')

**calculate vacors for each word**

In [ ]:
vocabulary =model.wv.vocab
# len(vocabulary)
# v=vocabulary['خوب'].

In [55]:
print(len(vocabulary),type(vocabulary))

966446 <class 'dict'>


In [56]:
vectors_arr=[]
for sent in cleaned_rokens_arr:
  vec_vocab=[model[word] for word in sent if word in vocabulary]# if model[word]
  vectors_arr.append(vec_vocab)

In [62]:
from sys import getsizeof
getsizeof(vectors_arr)

457360

In [ ]:
# vocabulary = word2vec.wv.vocab
# embedded_array=[]
# for sent in cleaned_rokens_arr:
#   vec_vocab=[word2vec.wv[word] for word in sent if word in vocabulary]
#   embedded_array.append(vec_vocab)

**save embedded words in csv file(too large!)>500mb**

In [66]:
# #save vctors to array
# import csv
# with open('vectors_arr.csv', 'w', newline='') as file:
#     mywriter = csv.writer(file, delimiter=',')
#     mywriter.writerows(vectors_arr)

# **Prepare Train data for train**

## **Vectors**

**lenght of vectors**

In [68]:
vectors_len = [len(x) for x in vectors_arr]
vectors=np.array(vectors_len)
max_vector_len=max(vectors_len)

**Add padding to short comemnts**

In [79]:
#add padding vetor 0
padding_vec=[0.0]*100
padding_vec=np.array(padding_vec)
pad_embedded_array=[]
for vector in vectors_arr:
  if len(vector)<max_vector_len:
    padding=[padding_vec]*(max_vector_len-len(vector))
    vector=vector+padding
  pad_embedded_array.append(vector)

In [ ]:
# import math
# padded_posts = []
# print(len(vec_vocab))
# for post in [vec_vocab]:
#     # Pad short posts with alternating min/max
#     if len(post) < 20:
#         # # Method 1
#         # pointwise_min = np.minimum.reduce(np.any(post, axis=1))
#         # m=min(post)
#         # pointwise_max = np.maximum.reduce(post)
#         # padding = [pointwise_max, pointwise_min]
#         # Method 2
#         pointwise_avg = np.mean(post)
#         padding = [pointwise_avg]
#         # post += padding * math.ceil((20 - len(post) / 2.0))
#         post += padding*(20-len(post))  
#     # Shorten long posts or those odd number length posts we padded to 51
#     if len(post) > 20:
#         post = post[:20]    
#     # Add the post to our new list of padded posts
#     padded_posts.append(post)

## **Labels**

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# Le = LabelEncoder()
# y = Le.fit_transform(new_df['sentiment'])

In [98]:
l=train_df["label_id"]

# **Traingin LSTM**

**Import tensowerflow**

In [83]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

In [82]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [84]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [99]:
from keras.layers import Embedding
from keras.layers import Dense, Input
from keras.layers import TimeDistributed
from keras.layers import LSTM, Bidirectional
from keras.models import Model
import tqdm
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
from tensorflow.python.keras.layers import Input
from keras.models import Sequential
# from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [100]:
pad_embedded_array.shape

AttributeError: ignored

**Create Model**

In [ ]:
# create the model
embedding_vecor_length = 100
lstm_model = Sequential()
# layer_1=
# layer_2=
# lstm_model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
# lstm_model.add(pad_embedded_array)
# lstm_model.add(LSTM(100))
# lstm_model.add(Dense(1, activation='sigmoid'))
# lstm_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
# model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

In [ ]:
#for test
# import pickle
# import sys
# import os
# from keras.preprocessing.sequence import pad_sequences
# from keras.utils.np_utils import to_categorical
# from keras.layers import TimeDistributed

# embedding_layer = Embedding(len(word2int) + 1,EMBEDDING_DIM,weights=[embedding_matrix],input_length=30,trainable=True)                         
# sequence_input = Input(shape=(30,), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
# l_lstm = Bidirectional(LSTM(69, return_sequences=True))(embedded_sequences)